In [21]:
import os, pandas as pd, numpy as np, torch, torchmetrics, pytorch_lightning as pl

from typing import Optional, Union

from torch.nn import functional as F
from torch import nn

from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from efficientnet_pytorch.model import EfficientNet

from kymatio.torch import TimeFrequencyScattering1D

from kymjtfs.batch_norm import ScatteringBatchNorm



class MedleySolosClassifier(LightningModule):
    def __init__(self, in_shape = 2**16, J = 12, Q = 16, F = 4, T = 2**11, lr=1e-3):
        super().__init__()

        self.in_shape = in_shape
        self.J = J
        self.Q = Q
        self.F = F
        self.T = T
        
        self.lr = lr
        
        self.s1_conv1 = nn.Sequential(
            Unsqueeze(1),
            nn.Conv2d(1, 4, kernel_size=(16, 1)),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=(4, 1), padding=(2, 0))
        )
        
        self.setup_jtfs()
        
        self.conv_net = EfficientNet.from_name('efficientnet-b0',
                                               in_channels=self.jtfs_channels,
                                               include_top = True,
                                               num_classes = 8)
        
        self.acc_metric = torchmetrics.AveragePrecision(num_classes = 8)

    def setup_jtfs(self):
        self.jtfs = TimeFrequencyScattering1D(
            shape=(self.in_shape, ),
            T=self.T,
            Q=self.Q,
            J=self.J,
            F=self.F,
            average_fr=True,
            max_pad_factor=1, 
            max_pad_factor_fr=1,
            out_3D=True,)
        
        n_channels = self._get_jtfs_out_dim()
        
        self.jtfs_dim = self._get_jtfs_out_dim()
        self.jtfs_channels = self.jtfs_dim[0]
        self.jtfs_bn = ScatteringBatchNorm(self.jtfs_dim)
        
    def forward(self, x):
        Sx = self.jtfs(x)
        
        s1, s2 = Sx[0], Sx[1]
        s1_conv = self.s1_conv1(s1)
        s1_conv = F.pad(s1_conv, 
                   (0, 0, s2.shape[-2] - s1_conv.shape[-2], 0))
        
        sx = torch.cat([s1_conv, s2], dim=1)[:, :, :32, :]
        sx = torch.log1p(self.jtfs_bn(sx))
        y = self.conv_net(sx)
        y = F.log_softmax(y, dim=1)
        return y
    
        
    def step(self, batch, fold):
        x, y = batch
        logits = self(x)

        loss, acc = F.nll_loss(logits, y), self.acc_metric(logits, y)
        
        self.log(f'{fold}/loss', loss, on_step=True, on_epoch=True, prog_bar=True)
#         self.log(f'{fold}/acc', acc, on_step=True, on_epoch=True, prog_bar=True)
        
#         return {f'loss': loss, f'{fold}/acc': acc}
        return {f'loss': loss}
    
    def log_metrics(self, outputs, fold):
        keys = list(outputs[0].keys())
        for k in keys:
            metric = torch.stack([x[k] for x in outputs]).mean()
            self.log(f'{fold}/{k}', metric)
        
    def training_step(self, batch, batch_idx):
        return self.step(batch, fold='train')

    def validation_step(self, batch, batch_idx):
        return self.step(batch, fold='val')

    def test_step(self, batch, batch_idx):
        return self.step(batch, fold='test')
    
    def configure_optimizers(self):
        opt = torch.optim.Adam(self.parameters(), lr=self.lr)
        return opt
    
    def _get_jtfs_out_dim(self):
        dummy_in = torch.randn(self.in_shape).cuda()
        sx = self.jtfs(dummy_in)
        s1 = self.s1_conv1(sx[0])
        s1 = F.pad(s1, (0, 0, sx[1].shape[-2] - s1.shape[-2], 0))
        S = torch.cat([s1, sx[1]], dim=1)[:, :, :32, :]
        out_dim = S.shape[1:3]
        return out_dim
    
class Unsqueeze(nn.Module):
    def __init__(self, dim=1):
        super(Unsqueeze, self).__init__()
        self.dim = dim

    def forward(self, x):
        return x.unsqueeze(self.dim)

### Data Loader

In [3]:
from torch.utils.data import Dataset, DataLoader
import mirdata.datasets.medley_solos_db as msdb



class MedleySolosDB(Dataset):
    def __init__(self, data_dir='/import/c4dm-datasets/medley-solos-db/', subset='training'):
        super().__init__()
        
        self.msdb = msdb.Dataset(data_dir)
        self.audio_dir = os.path.join(data_dir, 'audio')
        self.csv_dir = os.path.join(data_dir, 'annotation')
        self.subset = subset
        
        df = pd.read_csv(os.path.join(self.csv_dir, 'Medley-solos-DB_metadata.csv'))
        self.df = df.loc[df['subset'] == subset]
        self.df.reset_index(inplace = True)
        
    def build_audio_fname(self, df_item):
        uuid = df_item['uuid4']
        instr_id = df_item['instrument_id']
        subset = df_item['subset']
        return f'Medley-solos-DB_{subset}-{instr_id}_{uuid}.wav'

    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        audio_fname = self.build_audio_fname(item)
        audio, _ = msdb.load_audio(os.path.join(self.audio_dir, audio_fname))
        y = int(item['instrument_id'])
        
        return audio, y

    def __len__(self):
        return len(self.df)
        

class MedleyDataModule(pl.LightningDataModule):
    def __init__(self, 
                 data_dir: str = '/import/c4dm-datasets/medley-solos-db/', 
                 batch_size: int = 32):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size

    def setup(self, stage: Optional[str] = None):
        self.train_ds = MedleySolosDB(self.data_dir, subset='training')
        self.val_ds = MedleySolosDB(self.data_dir, subset='validation')
        self.test_ds = MedleySolosDB(self.data_dir, subset='test')

    def train_dataloader(self):
        return DataLoader(self.train_ds, batch_size=self.batch_size, shuffle=True, drop_last=True, num_workers=80)

    def val_dataloader(self):
        return DataLoader(self.val_ds, batch_size=self.batch_size, shuffle=False, drop_last=True, num_workers=80)

    def test_dataloader(self):
        return DataLoader(self.test_ds, batch_size=self.batch_size, shuffle=False, drop_last=True, num_workers=80)


### Mock Test

In [5]:
import os

train_loader = MedleySolosDB()
train_ds = DataLoader(train_loader, batch_size=64)
model = MedleySolosClassifier()
model(torch.randn(4, 2**16))

tensor([[-2.3772, -2.1299, -1.6823, -2.5611, -1.5263, -2.1659, -2.1802, -2.5235],
        [-2.2936, -1.7534, -2.0579, -1.8844, -2.2263, -1.9018, -2.4726, -2.2568],
        [-1.8175, -2.1123, -2.4692, -2.2959, -1.8422, -2.0866, -2.1370, -2.0349],
        [-2.1562, -2.6873, -2.1296, -2.8890, -2.5012, -1.5817, -1.7269, -1.7361]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward0>)

### Train and Eval

In [ ]:
from pytorch_lightning import Trainer

n_epochs, batch_size = 200, 4
early_stop_callback = EarlyStopping(monitor="val/loss", 
                                    min_delta=0.00, 
                                    patience=3, 
                                    verbose=False, 
                                    mode="max")
trainer = pl.Trainer(gpus=-1, 
                     max_epochs=n_epochs,
                     progress_bar_refresh_rate=1, 
                     checkpoint_callback=True,
                     callbacks=[early_stop_callback])
model, dataset = MedleySolosClassifier(), MedleyDataModule(batch_size=batch_size) 
trainer.fit(model, dataset)
trainer.test()

/homes/cv300/venvs/dafx/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:147: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=True)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=True)`.
  rank_zero_deprecation(
/homes/cv300/venvs/dafx/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:90: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=1)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


### Load from checkpoint

In [22]:
ckpt = '/homes/cv300/Documents/scattering/lightning_logs/version_15/checkpoints/epoch=9-step=14599.ckpt'
model = MedleySolosClassifier.load_from_checkpoint(ckpt)

/homes/cv300/venvs/dafx/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AveragePrecision` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [ ]:
from pytorch_lightning import Trainer

test_loader = MedleySolosDB(subset='test')
test_ds = DataLoader(test_loader, batch_size=4)

trainer = Trainer(gpus=-1, progress_bar_refresh_rate=1)
trainer.test(model, test_dataloaders=test_ds)

/homes/cv300/venvs/dafx/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:90: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=1)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/homes/cv300/venvs/dafx/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:906: LightningDeprecationWarning: `trainer.test(test_dataloaders)` is deprecated in v1.4 and will be removed in v1.6. Use `trainer.test(dataloaders)` instead.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [5]
/homes/cv300/venvs/dafx/lib/python3.8/site-packages/pytorch_lightning/trainer/data_

Testing: 0it [00:00, ?it/s]

/homes/cv300/venvs/dafx/lib/python3.8/site-packages/torchmetrics/functional/classification/average_precision.py:166: UserWarning: Average precision score for one or more classes was `nan`. Ignoring these classes in average
  warnings.warn(


In [14]:
len(test_loader)

12236

In [17]:
test_loader.df['instrument_id'].value_counts()

3    3167
7    2900
4    2609
2    1142
1     955
0     732
6     406
5     325
Name: instrument_id, dtype: int64